In [1]:
import pandas as pd
import json
import os
import bamboolib as bam

In [14]:
stacks = pd.read_parquet('../data/staging/mds/stack')

In [3]:
companies = pd.read_parquet('../data/staging/mds/company',use_nullable_dtypes = True)   

In [4]:
stacks.head()

                      catetory               company._id company.companyName  \
0               Data Streaming  62431d925695a66c21971421        Apache Kafka   
1               Event Tracking  62431d925695a66c21971438         Rudderstack   
2            Reverse ETL Tools  62431d925695a66c21971438         Rudderstack   
3  Customer Data Platform(CDP)  62431d925695a66c21971438         Rudderstack   
4                    ETL Tools  62431d925695a66c21971438         Rudderstack   

                                 company.companyLogo  company.slug  \
0  https://svn.apache.org/repos/asf/kafka/site/lo...  apache-kafka   
1    https://media.graphcms.com/PWlXzNpzQ7KI1PCLbJKV   rudderstack   
2    https://media.graphcms.com/PWlXzNpzQ7KI1PCLbJKV   rudderstack   
3    https://media.graphcms.com/PWlXzNpzQ7KI1PCLbJKV   rudderstack   
4    https://media.graphcms.com/PWlXzNpzQ7KI1PCLbJKV   rudderstack   

                 company.url  \
0  https://kafka.apache.org/   
1   https://rudderstack.com/   
2 

In [5]:
companies.head()

                        _id   companyName  \
0  62431d905695a66c2197123a           1mg   
1  62de961751b67be05f6baeb9  3step sports   
2  62b2f69d51b67be05f4acd54            5x   
3  62431d905695a66c2197123e      Adevinta   
4  62431d905695a66c219712b4        Affirm   

                                         description  \
0  1mg is an online platform that provides medica...   
1   step is the largest youth sport event and clu...   
2                     modern data stack as a service   
3  Adevinta is an online marketplace for second-h...   
4  Affirm modern technology to bring significant ...   

             organizationId  verified  
0  62431de75695a66c21974052     False  
1  62de961751b67be05f6baebc      True  
2  62b2f69d51b67be05f4acd57      True  
3  62431de75695a66c21974056     False  
4  62431de75695a66c219740cc     False  

### How many companies have added their stack?

In [6]:
companies['_id'].nunique()

359

In [17]:
len(stacks)

2140

### How many verified companies have added their stack?

In [7]:
companies.groupby('verified')["_id"].nunique()/len(companies)

verified
False    0.537604
True     0.454039
Name: _id, dtype: float64

In [8]:
import pandas as pd; import numpy as np
# Step: Group by verified and calculate new column(s)
companies = companies.groupby(['verified']).agg(frequency=('_id', 'size')).reset_index()

companies

   verified  frequency
0     False        193
1      True        163

### Some simple eda

In [18]:
import pandas as pd; import numpy as np
# Step: Group by and aggregate
stacks = stacks.groupby(['company.companyName']).agg(**{'company._id_size': ('company._id', 'size')}).reset_index()

# Step: Sort column(s) company._id_size descending (Z-A)
stacks = stacks.sort_values(by=['company._id_size'], ascending=[False])

stacks

    company.companyName  company._id_size
180                 dbt               220
151           Snowflake               145
65             Fivetran               123
100              Looker               119
6               Airflow               104
..                  ...               ...
122           PipeRider                 1
53              Datakin                 1
119               Okera                 1
118            Noteable                 1
93               Kestra                 1

[186 rows x 2 columns]

In [10]:
bb = stacks.groupby(["catetory","company.companyName"]).size().reset_index(name = 'counted')

KeyError: 'catetory'

In [ ]:
bb

In [ ]:
bb.sort_values(by = "counted",ascending=False ).head(50)

In [ ]:
import pandas as pd; import numpy as np
# Step: Group by and aggregate
gb_cat_comp_veri_unique = stacks.groupby(['catetory', 'company.companyName', 'verified']).agg(**{'company._id_size': ('company._id', 'size')}).reset_index()

# Step: Sort column(s) company._id_size descending (Z-A)
gb_cat_comp_veri_unique = gb_cat_comp_veri_unique.sort_values(by=['company._id_size'], ascending=[False])

gb_cat_comp_veri_unique

In [ ]:
# Step: Group by and aggregate
gb_cat_comp_veri_unique = stacks.groupby(['catetory', 'companyName', 'verified']).agg(**{'company._id_size': ('company._id', 'size')}).reset_index()



In [ ]:
gb_cat_comp_veri_unique

In [ ]:
import plotly.express as px
fig = px.histogram(gb_cat_comp_veri_unique, x='company.companyName', y='company._id_size', color='catetory')
fig